In [6]:
import requests
import pandas as pd

# CoinGecko API base URL
COINGECKO_API_URL = "https://api.coingecko.com/api/v3"

# Example token symbol (e.g., Ethereum)
TOKEN_ID = "ethereum"

# Fetch market data for the token
def get_market_data(token_id):
    url = f"{COINGECKO_API_URL}/coins/{token_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return {
            "current_price": data["market_data"]["current_price"]["usd"],
            "market_cap": data["market_data"]["market_cap"]["usd"],
            "volume_24h": data["market_data"]["total_volume"]["usd"],
            "circulating_supply": data["market_data"]["circulating_supply"],
        }
    else:
        print(f"Error fetching market data: {response.status_code}")
        return {}

# Etherscan API (replace YOUR_API_KEY with your actual key)
ETHERSCAN_API_URL = "https://api.etherscan.io/api"
ETHERSCAN_API_KEY = "QR4X5M7JHVBQ9XDXFHJIEUVBYUT9TDF4H6"

# Fetch transaction count for the token (as an example of activity)
def get_transaction_count(contract_address):
    url = f"{ETHERSCAN_API_URL}?module=account&action=tokentx&contractaddress={contract_address}&apikey={ETHERSCAN_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data.get("status") == "1":
            return len(data.get("result", []))  # Return count of transactions
        else:
            print(f"API error: {data.get('message')}")
            return 0
    else:
        print(f"Error fetching transaction data: {response.status_code}")
        return 0

# Model calculation of functionality coefficient (K_f)
def calculate_functionality_coeff(params):
    # Define weights for each functionality aspect
    weights = {
        "transaction_activity": 0.4,  # Weight for transaction activity
        "market_cap": 0.3,          # Weight for market capitalization
        "volume_24h": 0.2,          # Weight for 24h trading volume
        "circulating_supply": 0.1,  # Weight for circulating supply
    }

    # Normalize and calculate weighted sum
    max_value = max(params.values()) if max(params.values()) > 0 else 1  # Prevent division by zero
    normalized_params = {
        key: value / max_value for key, value in params.items()
    }
    k_f = sum(normalized_params[key] * weights[key] for key in weights)
    return k_f

# Main workflow
if __name__ == "__main__":
    # Fetch market data for Ethereum
    market_data = get_market_data(TOKEN_ID)

    if market_data:
        # Example transaction count (replace with real contract address if needed)
        contract_address = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"  # Replace with a real contract address
        transaction_activity = get_transaction_count(contract_address)

        # Combine parameters
        functionality_params = {
            "transaction_activity": transaction_activity,
            "market_cap": market_data.get("market_cap", 0),
            "volume_24h": market_data.get("volume_24h", 0),
            "circulating_supply": market_data.get("circulating_supply", 0),
        }

        # Calculate functionality coefficient
        k_f = calculate_functionality_coeff(functionality_params)

        # Output results
        print("Functionality Parameters:")
        print(pd.DataFrame([functionality_params]))
        print(f"\nFunctionality Coefficient (K_f): {k_f:.4f}")
    else:
        print("Failed to fetch market data.")



Functionality Parameters:
   transaction_activity    market_cap   volume_24h  circulating_supply
0                 10000  396492591496  32814597511        1.205023e+08

Functionality Coefficient (K_f): 0.3166
